In [1]:
import rpyc
import logging
import time
import cv2
import numpy as np
import base64
from IPython.display import display, Image  # No need for clear_output here
import ipywidgets as widgets
import os
import csv
import datetime
import torchvision.transforms as transforms
from PIL import Image



# --- Setup Logging ---
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('JetBotClient')

# --- Create Directories ---
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)

# --- Image Transformation ---
# Transformations *before* saving to disk (for consistency with training)
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


class RemoteJetBot:
    def __init__(self, ip_address, port=18861):
        logger.info(f"Connecting to JetBot at {ip_address}:{port}")
        try:
            self.conn = rpyc.connect(
                ip_address,
                port,
                config={
                    'sync_request_timeout': 30,
                    'allow_all_attrs': True
                }
            )
            logger.info("Connected successfully!")
            # Initialize video window
            self.image_widget = widgets.Image(
                format='jpeg',
                width=400,
                height=300,
            )
            display(self.image_widget)
        except Exception as e:
            logger.error(f"Connection failed: {str(e)}")
            raise

    def get_frame(self):
        """Get a single frame from the camera and display it"""
        try:
            # Get frame from server
            jpg_as_text = self.conn.root.get_camera_frame()
            if jpg_as_text:
                # Decode base64 string directly to bytes
                jpg_bytes = base64.b64decode(jpg_as_text)
                # Update the image widget
                self.image_widget.value = jpg_bytes

                # Convert to NumPy array (for saving)
                npimg = np.frombuffer(jpg_bytes, dtype=np.uint8)
                frame = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
                return frame  # Return the frame as a NumPy array
            return None

        except Exception as e:
            logger.error(f"Error getting frame: {str(e)}")
            return None

    def set_motors(self, left_speed, right_speed):
        try:
            logger.debug(f"Sending motor command: left={left_speed}, right={right_speed}")
            result = self.conn.root.set_motors(float(left_speed), float(right_speed))
            logger.debug("Command sent successfully")
            return result
        except Exception as e:
            logger.error(f"Error sending motor command: {str(e)}")
            raise

    def cleanup(self):
        try:
            logger.debug("Cleaning up connection")
            if hasattr(self, 'conn'):
                self.set_motors(0, 0)  # Stop motors
                self.conn.close()
            logger.info("Cleanup completed")
        except Exception as e:
            logger.error(f"Error during cleanup: {str(e)}")


def record_data(jetbot, duration, target_fps, motor_speed):
    """Records data from the Jetbot for a specified duration.

    Args:
        jetbot: The RemoteJetBot object.
        duration: Recording duration in seconds.
        target_fps: The desired frames per second.
        motor_speed:  The speed to set for the single motor.
    """

    with open(CSV_PATH, 'w', newline='') as csvfile:
        fieldnames = ['image_path', 'timestamp', 'action']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        target_interval = 1.0 / target_fps
        start_time = time.time()
        image_count = 0

        while time.time() - start_time < duration:
            frame_start_time = time.perf_counter()

            # 1. Capture image
            frame = jetbot.get_frame()  # Get frame as NumPy array
            if frame is None:  # Check if frame is valid
                continue

            # Convert to PIL Image
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # IMPORTANT: Convert to RGB
            image = Image.fromarray(frame)
            image = transform(image) # Apply the transforms

            # 2. Get motor speed (action)
            action = motor_speed  # The action is the constant motor speed

            # 3. Generate filename
            timestamp = time.time()
            image_filename = f"image_{image_count:05d}.jpg"
            image_path = os.path.join(IMAGE_DIR, image_filename)

            # 4. Save image
            pil_image = transforms.ToPILImage()((image + 1) / 2)  # Un-normalize
            pil_image.save(image_path)

            # 5. Write to CSV
            writer.writerow({'image_path': image_path, 'timestamp': timestamp, 'action': action})

            image_count += 1

            frame_end_time = time.perf_counter()
            elapsed_time = frame_end_time - frame_start_time
            sleep_time = target_interval - elapsed_time

            if sleep_time > 0:
                time.sleep(sleep_time)

    print(f"Data recording complete. {image_count} images saved.")




In [30]:

# --- Configuration ---
JETBOT_IP = '192.168.68.63'  # Replace with your Jetbot's IP address
DATA_DIR = 'jetbot_data'
IMAGE_DIR = os.path.join(DATA_DIR, 'images')
CSV_PATH = os.path.join(DATA_DIR, 'data.csv')
IMAGE_SIZE = 224  # Use 224x224 images
TARGET_FPS = 30
RECORDING_DURATION = 5  # in seconds
MOTOR_SPEED = 0.15  # Example speed.  Adjust as needed.

In [31]:
jetbot = RemoteJetBot(JETBOT_IP)

try:
    # Set the motor speed (single motor control)
    jetbot.set_motors(MOTOR_SPEED, 0)  # Left motor only

    # Record data
    record_data(jetbot, RECORDING_DURATION, TARGET_FPS, MOTOR_SPEED)

finally:
    jetbot.cleanup()  # Stop motors and close connection

INFO:JetBotClient:Connecting to JetBot at 192.168.68.63:18861
INFO:JetBotClient:Connected successfully!


Image(value=b'', format='jpeg', height='300', width='400')

DEBUG:JetBotClient:Sending motor command: left=0.15, right=0
DEBUG:JetBotClient:Command sent successfully
DEBUG:JetBotClient:Cleaning up connection
DEBUG:JetBotClient:Sending motor command: left=0, right=0
DEBUG:JetBotClient:Command sent successfully
INFO:JetBotClient:Cleanup completed


Data recording complete. 131 images saved.


In [21]:
jetbot = RemoteJetBot(JETBOT_IP)

INFO:JetBotClient:Connecting to JetBot at 192.168.68.63:18861
INFO:JetBotClient:Connected successfully!


Image(value=b'', format='jpeg', height='300', width='400')

In [22]:
jetbot.get_frame()

array([[[ 97,  88, 108],
        [ 90,  81, 101],
        [ 86,  77,  98],
        ...,
        [ 60,  69, 103],
        [ 56,  63,  96],
        [ 54,  61,  94]],

       [[ 95,  86, 106],
        [ 90,  81, 101],
        [ 85,  76,  97],
        ...,
        [ 57,  66,  99],
        [ 54,  61,  94],
        [ 52,  60,  90]],

       [[ 91,  82, 102],
        [ 88,  79,  99],
        [ 82,  73,  94],
        ...,
        [ 55,  64,  97],
        [ 52,  60,  90],
        [ 50,  58,  87]],

       ...,

       [[ 53,  56,  77],
        [ 57,  60,  81],
        [ 57,  62,  83],
        ...,
        [ 59,  60,  80],
        [ 59,  60,  80],
        [ 54,  55,  75]],

       [[ 55,  58,  79],
        [ 52,  55,  76],
        [ 53,  56,  77],
        ...,
        [ 57,  58,  79],
        [ 66,  67,  88],
        [ 56,  57,  78]],

       [[ 54,  57,  78],
        [ 54,  57,  78],
        [ 56,  59,  80],
        ...,
        [ 62,  63,  84],
        [ 61,  62,  83],
        [ 57,  58,  79]]

In [13]:
jetbot.set_motors(.115,0)

DEBUG:JetBotClient:Sending motor command: left=0.115, right=0
DEBUG:JetBotClient:Command sent successfully


True

In [14]:
jetbot.set_motors(0,0)

DEBUG:JetBotClient:Sending motor command: left=0, right=0
DEBUG:JetBotClient:Command sent successfully


True

In [23]:
jetbot.cleanup()

DEBUG:JetBotClient:Cleaning up connection
DEBUG:JetBotClient:Sending motor command: left=0, right=0
DEBUG:JetBotClient:Command sent successfully
INFO:JetBotClient:Cleanup completed
